# Snippet Collection

### Putty

In [ ]:
import subprocess

#subprocess.Popen('putty.exe -load "asl-client-1')
#subprocess.Popen('putty.exe -load "asl-mw-1')
#subprocess.Popen('putty.exe -load "asl-server-1')

In [ ]:
import subprocess

n_client = 3
n_middleware = 2
n_server = 3

for c in range(n_client):
    subprocess.Popen(f"putty.exe -load asl-client-{c+1}")

for mw in range(n_middleware):
    subprocess.Popen(f"putty.exe -load asl-mw-{mw+1}")
    
for s in range(n_server):
    subprocess.Popen(f"putty.exe -load asl-server-{s+1}")
    

### Azure

In [ ]:
list_vm_status()

In [ ]:
deallocate_all()

In [ ]:
import json
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.compute import ComputeManagementClient

def get_compute_client():
    with open("C:/Users/nicok/.ssh/asl/asl-experiment-master.json") as f:
        cred = json.load(f)

    subscription_id = cred['subscription_id']
    credentials = ServicePrincipalCredentials(
        client_id=cred['application_id'],
        secret=cred['key'],
        tenant=cred['tenant']
    )
    
    compute_client = ComputeManagementClient(credentials, subscription_id)
    return compute_client

def get_power_state(group_name, vm_name, compute_client=None):
    
    if compute_client is None:
        compute_client= get_compute_client()
    
    for status in compute_client.virtual_machines.instance_view(group_name, vm_name).statuses:
        if(status.code.startswith('PowerState/')):
            vm_status = status.code
    return vm_status

def list_vm_status(compute_client=None):
    
    if compute_client is None:
        compute_client= get_compute_client()
    
    for vm in compute_client.virtual_machines.list("asl"):
        vm_status = get_power_state("asl", vm.name, compute_client)
        print(f"{vm.name}: \t{vm_status}")
        
def deallocate_all(compute_client=None):
    
    if compute_client is None:
        compute_client= get_compute_client()
    
    async_vm_stops = []
    vm_names = []
    print("Deallocating all VM's...")
    
    for vm in compute_client.virtual_machines.list("asl"):
        async_vm_stop = compute_client.virtual_machines.deallocate("asl", vm.name)
        async_vm_stops.append(async_vm_stop)
        vm_names.append(vm.name)
        print(f"  Deallocating {vm.name}...")

    for vm_name, async_vm_stop in zip(vm_names, async_vm_stops):
        async_vm_stop.wait()
        print(f"  Finished Deallocating {vm_name}")
        
    print("Finished Deallocating all VM's")